# Creating system with MMElemental

In [1]:
#########################################
# Create ligand and receptor molecules #
#######################################

from mmelemental.models import Molecule
from mm_data import mols

# Construct MMSchema molecules
receptor = Molecule.from_file(mols["alanine.pdb"])
ligand   = Molecule.from_file(mols["ibu.pdb"])

In [2]:
import nglview

# Visualize ligand
nglview.show_mmelemental(ligand)

NGLWidget()

In [3]:
nglview.show_mmelemental(receptor)

NGLWidget()

# Generating ligand conformation with QCEgnine

In [4]:
import qcengine
import qcelemental as qcel

# Convert MMSchema to QCSchema molecule
qc_ligand = ligand.to_data("qcschema").data

# Create QCEngine input
qcin = {
    "molecule": qc_ligand,
    "driver": "energy",
    "model": {"method": "SCF", "basis": "sto-3g"},
}

# Run geometry optimization compute
qcout = qcengine.compute(qcin, program="nwchem")

# Extract new conformation
qc_ligand_opt = qcout.molecule

# Ibuprofen after optimization
nglview.show_qcelemental(qc_ligand_opt)

matched alpha and beta electrons 4 ('93', '33', '56', '0')
building CURRENT CORRELATION ENERGY ........................SUCCESS


NGLWidget()

# Running molecular docking with MMIC

In [5]:
import mmic_docking


# Convert QC-ligand to MMSchema mol
ligand_opt = Molecule.from_data(qc_ligand_opt, dtype="qcschema")

# Construct docking input data from MMSchema molecules
dockin = {
    "molecule": {
        "ligand": ligand_opt, 
        "receptor": receptor
    },
    "search_space": (-37.807, 5.045, -2.001, 30.131, -19.633, 37.987),
    "search_space_units": "angstrom",
    "component": "mmic_autodock_vina",
}

dockout = mmic_docking.components.DockComponent.compute(dockin)

scores, poses = dockout.scores, dockout.poses.ligand

print("Scores:")
print("=======")
print(scores)

print("\nLigand poses:")
print("=============")
print(poses)

Scores:
[-1.5, -1.5, -1.5, -1.4, -1.4, -1.3, -1.3, -1.2, -1.2]

Ligand poses:
[Molecule(name='C13HO2', hash='0d27403'), Molecule(name='C13HO2', hash='95adf91'), Molecule(name='C13HO2', hash='5effced'), Molecule(name='C13HO2', hash='bb1be35'), Molecule(name='C13HO2', hash='bb14b5b'), Molecule(name='C13HO2', hash='6a35714'), Molecule(name='C13HO2', hash='fb45205'), Molecule(name='C13HO2', hash='3b862de'), Molecule(name='C13HO2', hash='c817702')]
